# Notebook Instructions
<i>You can run the notebook document sequentially (one cell a time) by pressing <b> shift + enter</b>. While a cell is running, a [*] will display on the left. When it has been run, a number will display indicating the order in which it was run in the notebook [8].</i>

<i>Enter edit mode by pressing <b>`Enter`</b> or using the mouse to click on a cell's editor area. Edit mode is indicated by a green cell border and a prompt showing in the editor area.</i>

## Options Trading Strategy
Strategy: Short straddle before earnings announcement and square off after that.

### Define instrument name, earnings date, and strike difference

In [1]:
instrument_name = 'INFY'

strike_difference = 20

from datetime import date, timedelta
earnings_date_list = [date(2017,7,14), date(2017,10,24), date(2018,1,12), date(2018,4,13)]

### Strategy

In [2]:
from data import get_stock_data, get_option_data
from nsepy.derivatives import get_expiry_date
import mibian
from performance_analytics import compute_ret
import pandas as pd
import numpy as np

full_data = pd.Series()
for earnings_date in earnings_date_list:
    # Set start and end date
    start_date = earnings_date - timedelta(days=0)
    end_date = earnings_date + timedelta(days=3)    

    # Set ATM Strike Price
    close_price = get_stock_data(instrument_name,start_date,start_date).Close.values
    atm_strike_price = np.round(close_price/strike_difference*1.0,0)[0]*strike_difference

    # Get expiry date (you need to tweak the formula if earnings date falls after expiry date)
    expiry_date = get_expiry_date(year=end_date.year, month=end_date.month)
    
    # Get call and put option data
    data_ce = get_option_data(instrument_name,start_date,end_date,'CE',atm_strike_price,expiry_date)
    data_pe = get_option_data(instrument_name,start_date,end_date,'PE',atm_strike_price,expiry_date)
    
    # Calculate IVs
    days_to_expiry = (expiry_date - start_date).days
    Call_IV = mibian.BS([close_price, atm_strike_price, 0.05, days_to_expiry], callPrice=data_ce.Close.iloc[0]).impliedVolatility
    Put_IV = mibian.BS([close_price, atm_strike_price, 0.05, days_to_expiry], putPrice=data_pe.Close.iloc[0]).impliedVolatility
    
    # Generate Signal
    if Call_IV > 30 and Put_IV > 30:
        # Short Straddle
        data_pe['signal'] = -1
        data_ce['signal'] = -1
    else:
        # No Positions
        data_pe['signal'] = 0
        data_ce['signal'] = 0
        
    # Compute returns
    data_ce = compute_ret(data_ce)
    data_pe = compute_ret(data_pe)
    data =  (data_ce.strategy_ret + data_pe.strategy_ret)/2
    full_data = full_data.append(data*100)    

building dictionary


### Strategy performance

In [3]:
full_data

2017-07-14          NaN
2017-07-17     0.000000
2017-10-24          NaN
2017-10-25    51.713948
2017-10-26    20.740249
2018-01-12          NaN
2018-01-15    35.219266
2018-04-13          NaN
2018-04-16    17.980769
dtype: float64